In [2]:
from kafka_component.consumer import TweetConsumer, TickConsumer
from pyspark.sql import SparkSession
from pyspark.sql.functions import col  # 导入col用于引用DataFrame的列
from pyspark.sql.types import IntegerType  # 导入IntegerType用于列类型转换
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, HashingTF
from pyspark.ml.classification import LogisticRegression

In [4]:
# 初始化Spark
spark = SparkSession.builder.appName("ModelTraining").master("local[*]").getOrCreate()

# 读取大规模数据
data = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("./train_data.csv")

# 确保label列是整型
data = data.withColumn("label", col("label").cast(IntegerType()))

# 如果列名已经是message和label，下面的行可以省略
# data = data.withColumnRenamed("your_actual_text_column_name", "message")

# 确保只选择需要的列
data = data.select("message", "label")

# 创建NLP流水线
tokenizer = Tokenizer(inputCol="message", outputCol="words")
hashingTF = HashingTF(inputCol="words", outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.001)

pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

# 模型训练
model = pipeline.fit(data)

# 保存模型
model_path = "./pretrained_model"
model.save(model_path)
print("Model saved to " + model_path)

# 停止Spark
spark.stop()

24/05/29 16:12:11 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
24/05/29 16:12:13 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


Model saved to ./pretrained_model


In [3]:
model_path = "./pretrained_model"
consumer = TweetConsumer()
consumer.consume(topic_name='twitter', path_to_model=model_path)

your 131072x1 screen size is bogus. expect trouble
24/05/29 16:19:43 WARN Utils: Your hostname, LAPTOP-LSFQQOLG resolves to a loopback address: 127.0.1.1; using 172.30.10.193 instead (on interface eth0)
24/05/29 16:19:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/oufei/Conda/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/oufei/.ivy2/cache
The jars for the packages stored in: /home/oufei/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bf8f26a7-5ab5-4c2c-9105-669d8c1cf42d;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 252ms :: artifacts dl 8ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 from central in

-------------------------------------------
Batch: 0
-------------------------------------------
+---------------------------------------------------------------------------------------------------------------------------------------------------------+----------+
|message                                                                                                                                                  |prediction|
+---------------------------------------------------------------------------------------------------------------------------------------------------------+----------+
|{"message": "Just bought some more Ping An Bank stocks feeling pretty confident about its future growth!"}                                               |1.0       |
|{"message": "Ping An Bank's consistent performance over the years has made it a staple in my investment portfolio."}                                     |0.0       |
|{"message": "Feeling a bit disappointed with Ping An Bank's latest 

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/oufei/Conda/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
RuntimeError: reentrant call inside <_io.BufferedReader name=73>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/oufei/Conda/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/oufei/Conda/lib/python3.9/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/oufei/Conda/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "

Py4JError: An error occurred while calling o231.awaitTermination